In [126]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [74]:
def get_xt(df):
    columns = ['hr_state', 'sysbp_state', 'percoxyg_state', 'antibiotic_state', 'vaso_state', 'vent_state']
    categ_num = np.array([3, 3, 2, 2, 2, 2])
    sum_idx = 0
    prev_base = 1
    for i in range(len(columns)):
        idx = len(columns) - 1 - i
        sum_idx += prev_base * df[columns[idx]]
        prev_base *= categ_num[idx]
    df['X_t'] = sum_idx
    return df

In [75]:
observational_data = pd.read_csv("/data/localhost/taufiq/sontag-simulator-data/data.csv")
observational_data = get_xt(observational_data)

In [76]:
observational_data.head()

,hr_state,sysbp_state,percoxyg_state,glucose_state,antibiotic_state,vaso_state,vent_state,diabetic_idx,id,A_t,t,X_t
0,2,1,1,3,0,0,0,1,0,6,0,120
1,1,1,1,4,1,0,1,1,0,-1,1,77
2,1,1,1,1,0,0,0,0,1,4,0,72
3,1,1,1,2,1,0,0,0,1,-1,1,76
4,0,0,1,2,0,0,0,0,2,3,0,8


In [86]:
simulator_data = pd.read_csv("/data/localhost/taufiq/sontag-simulator-data/data_antibiotic2.csv").astype(int)

In [87]:
simulator_data.head()

,hr_state,sysbp_state,percoxyg_state,glucose_state,antibiotic_state,vaso_state,vent_state,diabetic_idx,id,t
0,0,0,1,1,0,0,0,1,0,0
1,0,0,1,1,1,0,1,1,0,1
2,2,0,1,1,0,0,0,1,1,0
3,1,0,1,2,1,1,1,1,1,1
4,1,2,1,1,0,0,0,1,2,0


In [108]:
def get_a0(df):
    actions = df[df['t']==1]
    actions.index = actions['id']
    actions = 4*actions['antibiotic_state'] + 2*actions['vent_state'] + actions['vaso_state']
    for index, row in df.iterrows():
        if row['t'] == 1:
            df.at[index, 'A_t'] = -1
        else:
            df.at[index, 'A_t'] = actions[row['id']]
    return df.astype(int)

In [112]:
simulator_data = get_a0(simulator_data)
simulator_data = get_xt(simulator_data)
simulator_data.head()

,hr_state,sysbp_state,percoxyg_state,glucose_state,antibiotic_state,vaso_state,vent_state,diabetic_idx,id,t,A_t,X_t
0,0,0,1,1,0,0,0,1,0,0,6,8
1,0,0,1,1,1,0,1,1,0,1,-1,13
2,2,0,1,1,0,0,0,1,1,0,7,104
3,1,0,1,2,1,1,1,1,1,1,-1,63
4,1,2,1,1,0,0,0,1,2,0,0,88


In [113]:
def get_sim_outcome(x, a, sim_data):
    ids = sim_data.loc[(sim_data['X_t']==x) & (sim_data['A_t']==a), 'id']
    df = sim_data[sim_data['id'].isin(ids)]
    hr = df.loc[df['t']==1, 'hr_state'].mean()
    sysbp = df.loc[df['t']==1, 'sysbp_state'].mean()
    percoxyg = df.loc[df['t']==1, 'percoxyg_state'].mean()
    return hr, sysbp, percoxyg

In [127]:
bounds = simulator_data.loc[simulator_data['t']==0 , ['A_t', 'X_t']].copy()
bounds.groupby(['A_t', 'X_t'])
bounds = bounds.reset_index(drop=True)
for index, row in tqdm(bounds.iterrows()):
    hr, sysbp, percoxyg = get_sim_outcome(row['X_t'], row['A_t'], simulator_data)
    bounds.at[index, '$E[hr^{sim}_{1}(a)|X_0 = x]$'] = hr
    bounds.at[index, '$E[sysbp^{sim}_{1}(a)|X_0 = x]$'] = sysbp
    bounds.at[index, '$E[percoxyg^{sim}_{1}(a)|X_0 = x]$'] = percoxyg    

10000it [00:19, 523.15it/s]


In [128]:
bounds.head()

,A_t,X_t,$E[hr^{sim}_{1}(a)|X_0 = x]$,$E[sysbp^{sim}_{1}(a)|X_0 = x]$,$E[percoxyg^{sim}_{1}(a)|X_0 = x]$
0,6,8,0.036364,0.400000,0.945455
1,7,104,1.441176,0.470588,0.955882
2,0,88,0.983471,1.752066,0.950413
3,4,120,1.473333,1.286667,0.920000
4,4,24,0.048276,1.296552,0.944828


In [138]:
def get_prob(x, a, obs_data):
    obs_filtered = obs_data.loc[(obs_data['X_t'] == x) & (obs_data['t'] == 0)]
    if len(obs_filtered) == 0:
        return float('NaN')
    return (obs_filtered['A_t']==a).sum()/len(obs_filtered)

In [139]:
def get_obs_bounds(x, a, obs_data):
    ids = obs_data.loc[(obs_data['X_t']==x) & (obs_data['A_t']==a), 'id']
    df = obs_data[obs_data['id'].isin(ids)]
    df = df[df['t']==1]
    hr_min = df['hr_state'].min()
    hr_max = df['hr_state'].max()
    hr_mean = df['hr_state'].mean()
    prob = get_prob(x, a, obs_data)
    lb = prob*(hr_mean) + (1-prob)*(hr_min)
    ub = prob*(hr_mean) + (1-prob)*(hr_max)
    return lb, ub

In [142]:
for index, row in tqdm(bounds.iterrows()):
    hr_lb, hr_ub = get_obs_bounds(row['X_t'], row['A_t'], observational_data)
    bounds.at[index, '$hr_{lb}$'] = hr_lb
    bounds.at[index, '$hr_{ub}$'] = hr_ub

10000it [00:22, 452.27it/s]


In [148]:
bounds.head()
bounds_dict = {
    
}

,A_t,X_t,$E[hr^{sim}_{1}(a)|X_0 = x]$,$E[sysbp^{sim}_{1}(a)|X_0 = x]$,$E[percoxyg^{sim}_{1}(a)|X_0 = x]$,$hr_{lb}$,$hr_{ub},$hr_{ub}$
0,6,8,0.036364,0.400000,0.945455,0.000000,0.000000,0.000000
1,7,104,1.441176,0.470588,0.955882,1.453515,1.496599,1.496599
2,0,88,0.983471,1.752066,0.950413,0.006948,0.998842,0.998842
3,4,120,1.473333,1.286667,0.920000,1.002368,1.996448,1.996448
4,4,24,0.048276,1.296552,0.944828,0.000000,0.000000,0.000000


In [147]:
def pr_hr_sim_not_in_bounds(obs_data, bounds):
    for index, row in obs_data.iterrows():
        

0.4586